# HATE SPEECH DETECTION

## DATA PREPROCESSING 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

In [4]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/home/spriyanshu723/prince/hate-speech-dataset/annotations_metadata.csv')

In [ ]:
data.head()

In [ ]:
path = '/home/spriyanshu723/prince/hate-speech-dataset/all_files/'

In [ ]:
def fileoutput(x):
    final = path + str(x) + '.txt'
    f = open(final, "r")
    string = ''
    string = f.read()
    #data['comment'] = string
    return string

In [ ]:
len(data)

In [ ]:
data.shape

In [ ]:
data['comment'] = 'hi'
data.head()

In [ ]:
for i in range(0,10944):
    data['comment'][i] = fileoutput(data['file_id'][i])

In [ ]:
data.head()

In [ ]:
with open('hate_speech_final_white', 'wb') as fp:
    pickle.dump(data, fp)

In [ ]:
with open('hate_speech_final_white', 'rb') as fp:
    data = pickle.load(fp)

In [ ]:
data.head(20)

In [ ]:
data = data.drop(['user_id'], axis=1)
data = data.drop(['subforum_id'], axis=1)
data = data.drop(['num_contexts'], axis=1)

In [ ]:
data = data[['file_id', 'comment', 'label']]

In [ ]:
data.head(20)

In [ ]:
data['comment'][10]

In [ ]:
data = data.drop(['file_id'], axis=1)
data.head()

In [ ]:
with open('hate_speech_final_white_one', 'wb') as fp:
    pickle.dump(data, fp)

In [5]:
with open('hate_speech_final_white_one', 'rb') as fp:
    data = pickle.load(fp)

In [6]:
data.head()

,comment,label
0,"As of March 13th , 2014 , the booklet had been...",noHate
1,In order to help increase the booklets downloa...,noHate
2,( Simply copy and paste the following text int...,noHate
3,Click below for a FREE download of a colorfull...,hate
4,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,noHate


In [7]:
data.shape

(10944, 2)

## RETRIVING TOP WORDS FROM HATE DATASET 

In [8]:
data_hate_three = data.loc[data['label'] == 'hate']

In [9]:
data_hate_three.head()

,comment,label
3,Click below for a FREE download of a colorfull...,hate
10,She may or may not be a Jew but she 's certain...,hate
21,I 've always considered teaching as one of the...,hate
22,And the sad thing is the white students at tho...,hate
31,Their gods often had pale skin and blue eyes b...,hate


In [10]:
data_hate_three.shape

(1196, 2)

In [11]:
data_unhate_three = data.loc[data['label'] == 'noHate']

In [12]:
data_unhate_three.head()

,comment,label
0,"As of March 13th , 2014 , the booklet had been...",noHate
1,In order to help increase the booklets downloa...,noHate
2,( Simply copy and paste the following text int...,noHate
4,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,noHate
5,"Booklet updated on Feb. 14th , 2014 .",noHate


In [13]:
data_unhate_three.shape

(9507, 2)

In [14]:
9507+1196

10703

In [15]:
data.label.unique()

array(['noHate', 'hate', 'idk/skip', 'relation'], dtype=object)

In [16]:
data_hate_three['class'] = 1

/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
data_hate_three = data_hate_three.drop(['label'], axis=1)
data_hate_three.head()

,comment,class
3,Click below for a FREE download of a colorfull...,1
10,She may or may not be a Jew but she 's certain...,1
21,I 've always considered teaching as one of the...,1
22,And the sad thing is the white students at tho...,1
31,Their gods often had pale skin and blue eyes b...,1


In [18]:
data_unhate_three['class'] = 0
data_unhate_three = data_unhate_three.drop(['label'], axis=1)
data_unhate_three.head()

/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,comment,class
0,"As of March 13th , 2014 , the booklet had been...",0
1,In order to help increase the booklets downloa...,0
2,( Simply copy and paste the following text int...,0
4,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,0
5,"Booklet updated on Feb. 14th , 2014 .",0


In [19]:
data_hate_three.shape

(1196, 2)

In [20]:
data_unhate_three.shape

(9507, 2)

In [21]:
9507 + 1196

10703

In [22]:
with open('a_hate_three', 'wb') as fp:
    pickle.dump(data_hate_three, fp)

In [23]:
with open('a_unhate_three', 'wb') as fp:
    pickle.dump(data_unhate_three, fp)

In [24]:
hate_df = data_hate_three['comment']
hate_df.head()

3     Click below for a FREE download of a colorfull...
10    She may or may not be a Jew but she 's certain...
21    I 've always considered teaching as one of the...
22    And the sad thing is the white students at tho...
31    Their gods often had pale skin and blue eyes b...
Name: comment, dtype: object

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
import nltk
from nltk.tokenize import TweetTokenizer 
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

In [28]:
tokenizer

In [33]:
one = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [34]:
one

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f2f8bb849d8>, use_idf=1,
        vocabulary=None)

In [35]:
trn_term_doc = one.fit_transform(hate_df)

In [36]:
trn_term_doc

<1196x2908 sparse matrix of type '<class 'numpy.float64'>'
	with 32729 stored elements in Compressed Sparse Row format>

In [37]:
indices_hate = np.argsort(one.idf_)[::-1]
features_hate = one.get_feature_names()
top_n = 100
top_features_hate = [features_hate[i] for i in indices_hate[:top_n]]
print(top_features_hate)

['less', 'somewhere', 'give them', 'some of them', 'girls are', 'get my', 'get away', 'something to', 'germans', 'some mongrel', 'german', 'sorry', 'genetic', 'sounds', 'gang', 'future of', 'some mongrel standing', 'giving me', 'future .', 'glassy - eyed', 'slowly', 'gods', "so it '", 'go back to', 'go .', 'global', 'glassy -', 'society .', 'so we', 'so you', 'social', 'glassy', 'giving me a', 'society ,', 'spend', 'spot', 'got in', 'study', 'for your', 'stormfront .', 'for white', 'straight', 'for this', 'students', 'for my', 'forced to', 'sub humans', 'for her', 'for a free', 'sun', 'followers', 'suppose they', 'forced', 'forcing', 'standing there', 'starting', 'standing there with', 'stare', 'stare .', 'from us', 'start a', 'from their', 'starting to', 'foreign', 'from the white', 'free download', 'state', 'stay here', 'steal', 'stick', 'good to', 'goyim .', 'the video', 'hero', 'sake', 'hiv / aids', 'hiv /', 'hispanics ,', 'higher', 'hey', 'here is', 'safe', 'here are', 'help them'

### SAVING TOP WORDS FROM HATE DATASET 

In [38]:
indices_hate = np.argsort(one.idf_)[::-1]
features_hate = one.get_feature_names()
top_n = 5000
top_features_hate = [features_hate[i] for i in indices_hate[:top_n]]

In [39]:
with open('z_hate_three', 'wb') as fp:
    pickle.dump(top_features_hate, fp)

## RETRIVING TOP WORDS FROM UNHATE DATASET 

In [40]:
data_unhate_three.head()

,comment,class
0,"As of March 13th , 2014 , the booklet had been...",0
1,In order to help increase the booklets downloa...,0
2,( Simply copy and paste the following text int...,0
4,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,0
5,"Booklet updated on Feb. 14th , 2014 .",0


In [41]:
unhate_df = data_unhate_three['comment']
unhate_df.head()

0    As of March 13th , 2014 , the booklet had been...
1    In order to help increase the booklets downloa...
2    ( Simply copy and paste the following text int...
4    Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...
5                Booklet updated on Feb. 14th , 2014 .
Name: comment, dtype: object

In [42]:
trn_term_doc = one.fit_transform(unhate_df)

In [43]:
trn_term_doc

<9507x17139 sparse matrix of type '<class 'numpy.float64'>'
	with 246372 stored elements in Compressed Sparse Row format>

In [44]:
indices_unhate = np.argsort(one.idf_)[::-1]
features_unhate = one.get_feature_names()
top_n = 100
top_features_unhate = [features_unhate[i] for i in indices_unhate[:top_n]]
print(top_features_unhate)

['just in case', 'posted .', 'posted a thread', 'posted on', 'posting it', 'posting the', 'posting your', 'posts ,', 'posts i', 'posts in', 'posts on', 'pot', 'color or', 'pound', 'pounds', 'color =', 'color ,', 'ppv', 'practice .', 'prayers', 'prayers go', 'college .', 'collective', 'pre - christian', 'preach', 'collection', 'predominantly', 'com / 2009', 'com / global', 'rifle and pistol', 'post that', 'coming up', 'pool', 'poor to', 'comes with', 'comes up', 'comes out', 'population ,', 'comes down to', 'comes down', 'populations .', 'portion of the', 'portugal', 'portuguese', 'come over', 'come out to', 'come out of', 'come out and', 'come across', 'combined', 'post ?', 'com |', 'post is', 'com / pages', 'post on here', 'post on this', 'prefer to', 'preference', 'preference of', 'preference of introversion', 'privilege .', 'clean their', 'classmates', 'pro white ,', 'classified', 'probably not', 'probation', 'classes .', 'claims that', 'claiming to', 'claimed that', 'claim that', '

### SAVING TOP WORDS FROM UNHATE DATASET 

In [45]:
indices_unhate = np.argsort(one.idf_)[::-1]
features_unhate = one.get_feature_names()
top_n = 5000
top_features_unhate = [features_unhate[i] for i in indices_unhate[:top_n]]

In [46]:
with open('z_unhate_three', 'wb') as fp:
    pickle.dump(top_features_unhate, fp)